<a href="https://colab.research.google.com/github/yebiny/SkillTreePython-DeepLearning/blob/main/04.Paper-with-code/comment/ch04_07_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ch04.07 Variational AutoEncoder

* 날짜:
* 이름:

## 학습내용
    - Variational AutoEncoder를 이해하고 구현한다.


In [1]:
import sys, os
sys.path.append('/content/drive/MyDrive/scripts/')
from lib import *

## VAE 기본 구현
---

![](https://user-images.githubusercontent.com/24144491/50323466-18d03700-051d-11e9-82ed-afb1b6e2666a.png)

### **| Encoder, Decoder 구현**

In [20]:
class Sampling(layers.Layer): # 상속

  def call(self, inputs):
    z_mu, z_sigma = inputs
    z_epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mu))
    z = ( tf.exp(z_epsilon*0.5) * z_sigma ) + z_mu
    return z

class BuildModel():
  def __init__(self, x_shape=(28,28,1), z_dims=100):
    self.x_shape = x_shape
    self.z_dims = z_dims

  def build_encoder(self):
    x = layers.Input(shape = self.x_shape )
    z = layers.Conv2D(32, 3, strides=2, padding='same', activation='leaky_relu')(x)
    z = layers.Conv2D(32, 3, strides=2, padding='same', activation='leaky_relu')(z)
    self.z_conv_shape = z.shape[1:] # (7, 7, 32)
 
    z = layers.Flatten()(z)
    z_mu = layers.Dense(z_dims, activation='relu')(z)
    z_sigma = layers.Dense(z_dims, activation='relu')(z)

    z = Sampling()([z_mu, z_sigma])
    encoder = models.Model(x, [z_mu, z_sigma, z], name='Encoder')
    return encoder

  def build_decoder(self):
    z = layers.Input(shape = self.z_dims)
    y = layers.Dense(256, activation='relu')(z)
    dim = self.z_conv_shape[0]*self.z_conv_shape[1]*self.z_conv_shape[2]
    y = layers.Dense(dim, activation = 'relu')(y)
    y = layers.Reshape(self.z_conv_shape)(y)
    y = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='leaky_relu')(y)
    y = layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation='sigmoid')(y)

    decoder = models.Model(z, y, name='Decoder')
    return decoder

In [21]:
x_shape = (28,28,1)
z_dims = 10
builder = BuildModel(x_shape, z_dims)
encoder = builder.build_encoder()
decoder = builder.build_decoder()
encoder.summary()
decoder.summary()

Model: "Encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 14, 14, 32)   320         ['input_14[0][0]']               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 7, 7, 32)     9248        ['conv2d_18[0][0]']              
                                                                                                  
 flatten_9 (Flatten)            (None, 1568)         0           ['conv2d_19[0][0]']              
                                                                                            

### **| 손실함수 정의**

![](https://user-images.githubusercontent.com/24144491/50323472-1a016400-051d-11e9-86b7-d8bf6a1a880f.png)

### **| VAE 구현**

### **| 학습**

### **| 결과 확인**